# Cell Segmentation
Segmentation of cell membranes in fluorescence microscopy images

In [1]:
import numpy as np
from skimage import io, filters, measure, segmentation
import matplotlib.pyplot as plt

## Load and preprocess image

In [2]:
# Generate a sample image for testing
img = np.zeros((100, 100))
img[20:40, 20:40] = 1
img[60:80, 60:80] = 1
img = filters.gaussian(img, sigma=1)
plt.imsave('preprocessed.png', img.astype(np.uint8), cmap='gray')

## Invert image and enhance dark cell interiors

In [3]:
inverted = 1 - img/img.max()
enhanced = filters.gaussian(inverted, sigma=3)
plt.imsave('enhanced.png', (enhanced*255).astype(np.uint8), cmap='gray')

## Detect cells using watershed

In [4]:
markers = filters.threshold_local(enhanced, 51) > 0
markers = measure.label(markers)
labels = segmentation.watershed(enhanced, markers)
plt.imsave('labels.png', labels.astype(np.uint8), cmap='nipy_spectral')
np.save('labels.npy', labels)